# Script 10: Encoding Categorico (Struttura Preservata) e Chiusura

In [ ]:
import pandas as pd

# Percorsi
input_path = "./data/monsters.csv"
output_path = "./data/monsters_final.csv"

# Caricamento
df = pd.read_csv(input_path)
print(f"✅ Dataset caricato: {input_path} ({df.shape})")


## 1. Rimozione Variabili ad Alta Cardinalità
Eliminiamo `sub_race` perché genererebbe troppe colonne sparse (centinaia di opzioni),
causando la "maledizione della dimensionalità".

In [ ]:
if 'sub_race' in df.columns:
    df = df.drop(columns=['sub_race'])
    print("✅ 'sub_race' rimossa (riduzione complessità).")


## 2. One-Hot Encoding di 'type' (Inserimento in-place)
Trasformiamo i 14 tipi di mostro in colonne binarie (type_Beast, type_Dragon...),
inserendole esattamente dove si trovava la colonna 'type' originale per mantenere l'ordine logico.

In [ ]:
if 'type' in df.columns:
    # 1. Trova posizione
    idx = df.columns.get_loc('type')
    
    # 2. Crea le colonne dummy (0/1)
    dummies = pd.get_dummies(df['type'], prefix='type', dtype=int)
    
    # 3. Inserisci le nuove colonne al posto della vecchia (Slicing + Concat)
    # Parte sinistra (prima di type) + Dummies + Parte destra (dopo type)
    df = pd.concat([df.iloc[:, :idx], dummies, df.iloc[:, idx+1:]], axis=1)
    
    print(f"✅ 'type' trasformato in {dummies.shape[1]} colonne binarie. Ordine preservato.")


## 3. Pulizia Finale (Safety Check)
Rimuoviamo qualsiasi colonna rimasta che non sia numerica (esclusa 'name').
Questo protegge da eventuali errori precedenti.

In [ ]:
# Seleziona colonne object (testo)
cols_text = df.select_dtypes(include=['object']).columns.tolist()

# 'name' deve rimanere, le altre via
cols_to_drop = [c for c in cols_text if c != 'name']

if cols_to_drop:
    df = df.drop(columns=cols_to_drop)
    print(f"⚠️ Rimossi residui testuali: {cols_to_drop}")
else:
    print("🎉 Check superato: Nessuna colonna testuale residua.")


## 4. Salvataggio Dataset Finale
Questo è il file che userai per l'addestramento dei modelli.

In [ ]:
try:
    df.to_csv(output_path, index=False)
    print(f"\n💾 DATASET PRONTO PER IL ML: {output_path}")
    print(f"   Shape finale: {df.shape}")
    
    # Anteprima (Colonne type)
    type_cols = [c for c in df.columns if 'type_' in c]
    print("\nEsempio trasformazione Type:")
    print(df[['name'] + type_cols[:4]].head(3)) # Mostra solo le prime 4 nuove colonne
except PermissionError:
    print("\n❌ ERRORE: Chiudi il file CSV aperto in Excel!")
